In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%%capture   
%pwd

In [3]:
%%capture
cd ..

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
import psycopg2 as ps2
import os
import pandas as pd
import numpy as np
import csv

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# pull in db credentials from .env file
db_name=os.getenv('DB_NAME')
db_user=os.getenv('DB_USER')
db_host=os.getenv('DB_HOST_IP')
db_pwd=os.getenv('DB_PWD')

In [12]:
# conn.close()
conn.rollback()

In [7]:
# define db session
conn = ps2.connect(
    host = db_host,
    database = db_name,
    user = db_user,
    password = db_pwd)

### Load Data

In [6]:
%%time

df_rel=pd.read_pickle('./data/processed/directional_df.pkl')
df_rel.info(); df_rel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26547903 entries, 0 to 26547902
Data columns (total 15 columns):
 #   Column            Dtype   
---  ------            -----   
 0   :START_ID         int64   
 1   type:TYPE         category
 2   :END_ID           int64   
 3   ontology          category
 4   effect            category
 5   mechanism         category
 6   ref_count:int     int8    
 7   biomarkertype     category
 8   celllinename      category
 9   celltype          category
 10  changetype        category
 11  organ             category
 12  organism          category
 13  quantitativetype  category
 14  tissue            category
dtypes: category(12), int64(2), int8(1)
memory usage: 835.6 MB
Wall time: 943 ms


,:START_ID,type:TYPE,:END_ID,ontology,effect,mechanism,ref_count:int,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue
0,-4129340824583713637,Regulation,-2983484129356197270,None,None,None,1,None,None,None,None,intestine,Rattus norvegicus,None,None
1,7111680438658927661,Regulation,-3121142714757276759,None,None,None,1,None,None,None,None,None,None,None,None
2,-8447327320654612427,Regulation,-1646880163763186214,None,None,None,10,None,TOV-112D,None,None,None,None,None,None
3,-8447327320654612427,Regulation,-1646880163763186214,None,None,None,10,None,None,B-cell,None,None,None,None,None
4,-8447327320654612427,Regulation,-1646880163763186214,None,None,None,10,None,None,endothelial cell,None,None,None,None,None


In [ ]:
# df_rel['mechanism_']=df_rel[[':START_ID', 'type:TYPE', ':END_ID', 'mechanism']].groupby([':START_ID', 'type:TYPE', ':END_ID'])['mechanism'].apply(set)

In [ ]:
# df_rel[[':START_ID', 'type:TYPE', ':END_ID', 'mechanism_']].head()

In [8]:
# define query
# sql="SELECT DISTINCT \
control.id, inkey[1], controltype, ontology, relationship, effect, mechanism, num_refs, outkey[1] , reference.id, biomarkertype, celllinename, celltype, \
changetype, organ, organism, quantitativetype, tissue  \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null"

In [13]:
# define query
sql="SELECT DISTINCT \
control.id, inkey[1], controltype,  string_agg(distinct(effect), ','), string_agg(distinct(mechanism), ','), \
num_refs, outkey[1] , reference.id, string_agg(distinct(biomarkertype), ',') , string_agg(distinct(celllinename), ','), \
string_agg(distinct(celltype), ','), string_agg(distinct(changetype), ','), string_agg(distinct(organ), ','), \
string_agg(distinct(organism), ','), string_agg(distinct(quantitativetype), ','), string_agg(distinct(tissue), ','),  \
string_agg(distinct(nct_id), ','),  string_agg(distinct(phase), ',') \
FROM resnet.control, resnet.reference \
WHERE control.id = reference.id and inkey[1] is not null and outkey[1] is not null \
GROUP BY control.id, inkey[1], controltype, outkey[1], reference.id"

In [14]:
%%time
# run query and save to file
with conn.cursor() as cur:
    with open('./data/interim/relations_query_csl_test.txt', 'w', encoding="utf-8") as f:
        cur.execute(sql)
        csv_writer=csv.writer(f, delimiter= '|')
        for record in cur.fetchall():
            line='|'.join(map(str, record))+'\n'
            f.write(line)

Wall time: 19min 54s


In [15]:
df=pd.read_csv('./data/interim/relations_query_csl_test.txt', sep='|', header=None, encoding='utf-8')

In [ ]:
df.info(); df.head()

In [17]:
df.to_pickle('./data/interim/relations_query_csl_test.pkl')

In [18]:
df=pd.read_pickle('./data/interim/relations_query_csl_test.pkl')

In [26]:
df.columns=['id1', ':START_ID', 'type:TYPE', 'effect', 'mechanism', 'ref_count:int', ':END_ID' , 'id2', \
            'biomarkertype', 'celllinename', 'celltype', 'changetype', 'organ', 'organism', 'quantitativetype', \
            'tissue', 'nct_id', 'phase']

In [27]:
df.info(); df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10191129 entries, 0 to 10191128
Data columns (total 18 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   id1               int64 
 1   :START_ID         int64 
 2   type:TYPE         object
 3   effect            object
 4   mechanism         object
 5   ref_count:int     int64 
 6   :END_ID           int64 
 7   id2               int64 
 8   biomarkertype     object
 9   celllinename      object
 10  celltype          object
 11  changetype        object
 12  organ             object
 13  organism          object
 14  quantitativetype  object
 15  tissue            object
 16  nct_id            object
 17  phase             object
dtypes: int64(5), object(13)
memory usage: 1.4+ GB


,id1,:START_ID,type:TYPE,effect,mechanism,ref_count:int,:END_ID,id2,biomarkertype,celllinename,celltype,changetype,organ,organism,quantitativetype,tissue,nct_id,phase
0,-9223365986387833915,-4129340824583713637,Regulation,None,None,1,-2983484129356197270,-9223365986387833915,None,None,None,None,intestine,Rattus norvegicus,None,None,None,None
1,-9223365556180770043,7111680438658927661,Regulation,None,None,1,-3121142714757276759,-9223365556180770043,None,None,None,None,None,None,None,None,None,None
2,-9223365466130767478,-8447327320654612427,Regulation,None,None,10,-1646880163763186214,-9223365466130767478,None,TOV-112D,"B-cell,endothelial cell",None,peripheral nervous system,None,None,None,None,None
3,-9223362118268724280,3670031740130349683,Regulation,positive,None,1,-4650140568030604438,-9223362118268724280,None,None,plasma cell,None,None,None,None,plasma,None,None
4,-9223361192663839079,-6414782697893422681,Expression,positive,None,1,-296331965523681380,-9223361192663839079,None,None,T-cell,None,None,None,None,None,None,None


In [31]:
df['biomarkertype'].value_counts()

None                     10051277
diagnostic                  70971
prognostic                  49620
diagnostic,prognostic       19261
Name: biomarkertype, dtype: int64

In [24]:
df.to_pickle('./data/interim/relations_query_csl_test.pkl')